# Praktikum Minggu 15: Studi Kasus End-to-End — Prediksi Churn Pelanggan
## *Week 15 Lab: End-to-End Case Study — Customer Churn Prediction*

**Mata Kuliah / Course:** Big Data Analytics  
**Topik / Topic:** CRISP-DM, EDA, Feature Engineering, ML Pipeline, Deployment  

---
### Deskripsi
Praktikum ini menerapkan metodologi CRISP-DM secara end-to-end untuk memprediksi
churn pelanggan telekomunikasi — salah satu use case paling umum di industri.
Kita akan melalui seluruh siklus proyek data analytics dari business understanding
hingga simulasi deployment.

In [ ]:
# All imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, precision_recall_curve
)
import joblib

np.random.seed(42)
print('Libraries loaded successfully!')

## Studi Kasus: Prediksi Churn Pelanggan Telekomunikasi

**Churn** (customer attrition) adalah ketika pelanggan berhenti menggunakan layanan.
Biaya akuisisi pelanggan baru 5–25× lebih mahal daripada mempertahankan pelanggan lama.
Mengidentifikasi pelanggan berisiko churn lebih awal sangat bernilai bagi bisnis.

In [ ]:
# Generate realistic telecom churn dataset (1000 customers)
def generate_telecom_dataset(n=1000, seed=42):
    np.random.seed(seed)

    # Features
    tenure = np.random.exponential(scale=24, size=n).astype(int).clip(1, 72)
    contract_type = np.random.choice(['Month-to-Month', 'One Year', 'Two Year'],
                                      size=n, p=[0.55, 0.25, 0.20])
    payment_method = np.random.choice(
        ['Electronic Check', 'Mailed Check', 'Bank Transfer', 'Credit Card'],
        size=n, p=[0.35, 0.25, 0.22, 0.18]
    )
    num_services   = np.random.randint(1, 7, size=n)
    monthly_charges = 20 + num_services * 8 + np.random.normal(0, 5, n)
    monthly_charges = monthly_charges.clip(18, 120).round(2)
    total_charges   = (tenure * monthly_charges + np.random.normal(0, 20, n)).clip(0).round(2)
    cust_service_calls = np.random.poisson(1.5, n).clip(0, 10)

    # Churn probability (business logic)
    churn_score = (
        0.40 * (contract_type == 'Month-to-Month').astype(int)
        - 0.20 * (tenure > 24).astype(int)
        + 0.15 * (cust_service_calls > 3).astype(int)
        + 0.10 * (payment_method == 'Electronic Check').astype(int)
        - 0.10 * (num_services > 4).astype(int)
        + np.random.normal(0, 0.15, n)
    )
    churn_prob = 1 / (1 + np.exp(-churn_score * 2))
    churn = (churn_prob > 0.5).astype(int)

    return pd.DataFrame({
        'customer_id': [f'CUST_{i:04d}' for i in range(1, n+1)],
        'tenure': tenure,
        'contract_type': contract_type,
        'payment_method': payment_method,
        'num_services': num_services,
        'monthly_charges': monthly_charges,
        'total_charges': total_charges,
        'customer_service_calls': cust_service_calls,
        'churn': churn
    })

df = generate_telecom_dataset(n=1000)
print(f'Dataset shape: {df.shape}')
print(f'Churn rate: {df["churn"].mean():.2%}')
print('\nFirst 5 rows:')
print(df.head())
print('\nData types & nulls:')
print(df.info())

## 1. Business Understanding (CRISP-DM)

Mendefinisikan masalah bisnis secara jelas sebelum mulai menganalisis data.

In [ ]:
business_problem = """
╔══════════════════════════════════════════════════════════════════════════╗
║                 BUSINESS UNDERSTANDING — CRISP-DM PHASE 1               ║
╠══════════════════════════════════════════════════════════════════════════╣
║                                                                          ║
║  MASALAH BISNIS:                                                         ║
║  TelcoIndo mengalami churn rate 26% per kuartal. Setiap pelanggan        ║
║  yang churn merepresentasikan kerugian rata-rata Rp 2.4 juta/tahun.     ║
║  Dengan 50.000 pelanggan aktif, churn 26% = 13.000 pelanggan hilang      ║
║  = kerugian Rp 31.2 miliar/tahun.                                        ║
║                                                                          ║
║  TUJUAN ANALITIK:                                                        ║
║  Membangun model prediksi churn dengan recall ≥ 70% untuk                ║
║  mengidentifikasi pelanggan berisiko tinggi 30 hari sebelum churn.       ║
║                                                                          ║
║  STRATEGI INTERVENSI:                                                    ║
║  - High-risk (score > 0.7): Proactive outreach + retention offer         ║
║  - Medium-risk (0.4-0.7): Loyalty reward program                         ║
║  - Low-risk (< 0.4): Standard retention program                          ║
║                                                                          ║
║  SUCCESS CRITERIA:                                                       ║
║  - Model recall ≥ 70% (minimize false negatives)                         ║
║  - ROC-AUC ≥ 0.75                                                        ║
║  - Reduce churn by 15% within 6 months post-deployment                   ║
╚══════════════════════════════════════════════════════════════════════════╝
"""
print(business_problem)

## 2. Data Understanding — EDA

Eksplorasi data secara menyeluruh: distribusi, korelasi, dan pola churn.

In [ ]:
print('=== Basic Statistics ===')
print(df.describe().round(2))

fig = plt.figure(figsize=(16, 12))
gs = gridspec.GridSpec(3, 3, figure=fig)

# 1. Churn distribution
ax1 = fig.add_subplot(gs[0, 0])
churn_counts = df['churn'].value_counts()
ax1.pie(churn_counts, labels=['No Churn', 'Churn'], autopct='%1.1f%%',
        colors=['steelblue', 'tomato'], startangle=90)
ax1.set_title('Churn Distribution')

# 2. Tenure distribution by churn
ax2 = fig.add_subplot(gs[0, 1])
df[df['churn']==0]['tenure'].hist(bins=20, ax=ax2, alpha=0.6, color='steelblue', label='No Churn')
df[df['churn']==1]['tenure'].hist(bins=20, ax=ax2, alpha=0.6, color='tomato', label='Churn')
ax2.set_title('Tenure Distribution by Churn'); ax2.legend()

# 3. Monthly charges by churn
ax3 = fig.add_subplot(gs[0, 2])
df.boxplot(column='monthly_charges', by='churn', ax=ax3)
ax3.set_title('Monthly Charges by Churn'); ax3.set_xlabel('Churn (0=No, 1=Yes)')
plt.sca(ax3)  # suppress automatic figure title

# 4. Churn rate by contract type
ax4 = fig.add_subplot(gs[1, 0])
churn_by_contract = df.groupby('contract_type')['churn'].mean().sort_values(ascending=False)
churn_by_contract.plot(kind='bar', ax=ax4, color='coral', edgecolor='white')
ax4.set_title('Churn Rate by Contract Type'); ax4.set_ylabel('Churn Rate')
ax4.tick_params(axis='x', rotation=20)
for bar, val in zip(ax4.patches, churn_by_contract):
    ax4.text(bar.get_x()+0.2, bar.get_height()+0.01, f'{val:.1%}', fontsize=9)

# 5. Churn rate by payment method
ax5 = fig.add_subplot(gs[1, 1])
churn_by_pay = df.groupby('payment_method')['churn'].mean().sort_values(ascending=False)
churn_by_pay.plot(kind='bar', ax=ax5, color='mediumpurple', edgecolor='white')
ax5.set_title('Churn Rate by Payment Method'); ax5.set_ylabel('Churn Rate')
ax5.tick_params(axis='x', rotation=30)

# 6. Customer service calls vs churn
ax6 = fig.add_subplot(gs[1, 2])
churn_by_calls = df.groupby('customer_service_calls')['churn'].mean()
ax6.bar(churn_by_calls.index, churn_by_calls.values, color='sandybrown')
ax6.set_title('Churn Rate by Service Calls'); ax6.set_xlabel('# Service Calls')
ax6.set_ylabel('Churn Rate')

# 7. Correlation heatmap (numeric only)
ax7 = fig.add_subplot(gs[2, :])
numeric_cols = ['tenure', 'num_services', 'monthly_charges', 'total_charges',
                'customer_service_calls', 'churn']
corr_matrix = df[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', ax=ax7,
            square=True, linewidths=0.5)
ax7.set_title('Feature Correlation Heatmap')

fig.suptitle('Exploratory Data Analysis — Telecom Churn Dataset', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Key findings
print('\n=== Key EDA Findings ===')
print(f'- Overall churn rate: {df["churn"].mean():.1%}')
print(f'- Month-to-Month churn rate: {df[df["contract_type"]=="Month-to-Month"]["churn"].mean():.1%}')
print(f'- Two Year contract churn rate: {df[df["contract_type"]=="Two Year"]["churn"].mean():.1%}')
print(f'- Avg tenure churners: {df[df["churn"]==1]["tenure"].mean():.1f} months')
print(f'- Avg tenure non-churners: {df[df["churn"]==0]["tenure"].mean():.1f} months')

## 3. Data Preparation

Preprocessing pipeline: encoding kategoris, scaling numerik, train/test split.

In [ ]:
# Features and target
feature_cols = ['tenure', 'contract_type', 'payment_method', 'num_services',
                'monthly_charges', 'total_charges', 'customer_service_calls']
target_col = 'churn'

X = df[feature_cols].copy()
y = df[target_col].copy()

# Identify numeric and categorical columns
numeric_features = ['tenure', 'num_services', 'monthly_charges',
                    'total_charges', 'customer_service_calls']
categorical_features = ['contract_type', 'payment_method']

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Train/test split (stratified to preserve churn ratio)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Train size: {X_train.shape[0]} | Test size: {X_test.shape[0]}')
print(f'Train churn rate: {y_train.mean():.2%} | Test churn rate: {y_test.mean():.2%}')

# Fit preprocessor
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc  = preprocessor.transform(X_test)

# Get feature names after preprocessing
cat_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
all_feature_names = numeric_features + list(cat_feature_names)
print(f'\nProcessed feature count: {X_train_proc.shape[1]}')
print(f'Feature names: {all_feature_names}')

## 4. Modeling — Beberapa Algoritma

Melatih dan membandingkan tiga algoritma: Logistic Regression, Random Forest, Gradient Boosting.

In [ ]:
# Define models
models_dict = {
    'Logistic Regression': LogisticRegression(max_iter=500, random_state=42, C=1.0),
    'Random Forest':       RandomForestClassifier(n_estimators=100, random_state=42, max_depth=8),
    'Gradient Boosting':   GradientBoostingClassifier(n_estimators=100, random_state=42, learning_rate=0.1)
}

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}

print('=== Cross-Validation Results (5-fold Stratified) ===')
for name, model in models_dict.items():
    scores = cross_val_score(model, X_train_proc, y_train, cv=cv,
                              scoring='roc_auc', n_jobs=-1)
    cv_results[name] = scores
    print(f'{name:25s}: AUC = {scores.mean():.4f} ± {scores.std():.4f}')

# Train all models on full training set
trained_models = {}
for name, model in models_dict.items():
    model.fit(X_train_proc, y_train)
    trained_models[name] = model

# Evaluate on test set
print('\n=== Test Set Evaluation ===')
results_df_rows = []
for name, model in trained_models.items():
    y_pred = model.predict(X_test_proc)
    y_proba = model.predict_proba(X_test_proc)[:, 1]
    results_df_rows.append({
        'Model': name,
        'Accuracy': round(accuracy_score(y_test, y_pred), 4),
        'Precision': round(precision_score(y_test, y_pred), 4),
        'Recall': round(recall_score(y_test, y_pred), 4),
        'F1-Score': round(f1_score(y_test, y_pred), 4),
        'ROC-AUC': round(roc_auc_score(y_test, y_proba), 4)
    })

results_df = pd.DataFrame(results_df_rows).set_index('Model')
print(results_df.to_string())

# Select best model by ROC-AUC
best_model_name = results_df['ROC-AUC'].idxmax()
best_model = trained_models[best_model_name]
print(f'\n✅ Best model: {best_model_name} (AUC = {results_df.loc[best_model_name, "ROC-AUC"]:.4f})')

## 5. Evaluation

Evaluasi mendalam model terbaik: confusion matrix, ROC curve, feature importance.

In [ ]:
y_pred_best  = best_model.predict(X_test_proc)
y_proba_best = best_model.predict_proba(X_test_proc)[:, 1]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Confusion Matrix
ax = axes[0]
cm = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['No Churn', 'Churn'],
            yticklabels=['No Churn', 'Churn'])
ax.set_title(f'{best_model_name}\nConfusion Matrix')
ax.set_ylabel('Actual'); ax.set_xlabel('Predicted')

# 2. ROC Curves (all models)
ax = axes[1]
colors = ['steelblue', 'tomato', 'seagreen']
for (name, model), color in zip(trained_models.items(), colors):
    y_proba = model.predict_proba(X_test_proc)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    ax.plot(fpr, tpr, color=color, label=f'{name} (AUC={auc:.3f})')
ax.plot([0,1],[0,1], 'k--', alpha=0.5)
ax.set_title('ROC Curves — All Models')
ax.set_xlabel('False Positive Rate'); ax.set_ylabel('True Positive Rate')
ax.legend(fontsize=8); ax.grid(alpha=0.3)

# 3. Feature Importance (for tree-based models)
ax = axes[2]
if hasattr(best_model, 'feature_importances_'):
    importance = pd.Series(best_model.feature_importances_, index=all_feature_names)
    importance.sort_values(ascending=True).tail(10).plot(kind='barh', ax=ax, color='steelblue')
    ax.set_title(f'{best_model_name}\nTop Feature Importances')
    ax.set_xlabel('Importance')
elif hasattr(best_model, 'coef_'):
    coef = pd.Series(np.abs(best_model.coef_[0]), index=all_feature_names)
    coef.sort_values(ascending=True).tail(10).plot(kind='barh', ax=ax, color='coral')
    ax.set_title(f'{best_model_name}\n|Coefficient| — Feature Importance')

plt.tight_layout()
plt.show()

print(f'\n=== {best_model_name} — Classification Report ===')
print(classification_report(y_test, y_pred_best, target_names=['No Churn', 'Churn']))

## 6. Deployment Simulation

Mensimulasikan deployment: menyimpan model, memuat kembali, dan membuat prediksi pada data baru.

In [ ]:
import os

# Save model and preprocessor
model_artifacts = {
    'model': best_model,
    'preprocessor': preprocessor,
    'feature_cols': feature_cols,
    'model_name': best_model_name
}
joblib.dump(model_artifacts, 'churn_model.pkl')
file_size = os.path.getsize('churn_model.pkl') / 1024
print(f'Model saved to churn_model.pkl ({file_size:.1f} KB)')

# Load model back
loaded = joblib.load('churn_model.pkl')
loaded_model = loaded['model']
loaded_preprocessor = loaded['preprocessor']
print(f'Model loaded: {loaded["model_name"]}')

# Simulate new customer predictions
new_customers = pd.DataFrame([
    {'tenure': 2, 'contract_type': 'Month-to-Month', 'payment_method': 'Electronic Check',
     'num_services': 2, 'monthly_charges': 65.0, 'total_charges': 130.0, 'customer_service_calls': 5},
    {'tenure': 36, 'contract_type': 'Two Year', 'payment_method': 'Credit Card',
     'num_services': 5, 'monthly_charges': 85.0, 'total_charges': 3060.0, 'customer_service_calls': 0},
    {'tenure': 12, 'contract_type': 'One Year', 'payment_method': 'Bank Transfer',
     'num_services': 3, 'monthly_charges': 55.0, 'total_charges': 660.0, 'customer_service_calls': 2},
    {'tenure': 1, 'contract_type': 'Month-to-Month', 'payment_method': 'Mailed Check',
     'num_services': 1, 'monthly_charges': 25.0, 'total_charges': 25.0, 'customer_service_calls': 4},
])

X_new = loaded_preprocessor.transform(new_customers)
predictions = loaded_model.predict(X_new)
probabilities = loaded_model.predict_proba(X_new)[:, 1]

print('\n=== New Customer Predictions ===')
print(f'{"Customer":<10} {"Churn Prob":>12} {"Prediction":>12} {"Risk Level":>12}')
print('-' * 50)
for i, (pred, prob) in enumerate(zip(predictions, probabilities)):
    risk = 'HIGH' if prob > 0.7 else ('MEDIUM' if prob > 0.4 else 'LOW')
    print(f'Customer {i+1:<2} {prob:>12.3f} {"Churn" if pred else "No Churn":>12} {risk:>12}')

os.remove('churn_model.pkl')
print('\nCleanup: churn_model.pkl removed.')

## 7. Business Insights & Rekomendasi

Menerjemahkan temuan model menjadi rekomendasi bisnis yang actionable.

In [ ]:
# Quantify business impact
n_test = len(y_test)
actual_churners = y_test.sum()
detected_churners = ((y_pred_best == 1) & (y_test == 1)).sum()  # True Positives
recall_val = recall_score(y_test, y_pred_best)

REVENUE_PER_CUSTOMER = 2_400_000  # Rp per year
INTERVENTION_COST    = 200_000    # Rp per customer (retention offer)
RETENTION_RATE       = 0.35       # 35% of identified at-risk customers retained

retained_customers = int(detected_churners * RETENTION_RATE)
revenue_saved  = retained_customers * REVENUE_PER_CUSTOMER
intervention_cost = detected_churners * INTERVENTION_COST
net_benefit = revenue_saved - intervention_cost

insights = f"""
╔═══════════════════════════════════════════════════════════════════╗
║               BUSINESS INSIGHTS & REKOMENDASI                    ║
╠═══════════════════════════════════════════════════════════════════╣
║                                                                   ║
║  MODEL PERFORMANCE (Test Set: {n_test} customers):                ║
║  - Actual churners:     {actual_churners:>4}                               ║
║  - Detected by model:   {detected_churners:>4} (Recall: {recall_val:.1%})                ║
║  - Missed churners:     {actual_churners - detected_churners:>4}                               ║
║                                                                   ║
║  PERKIRAAN DAMPAK BISNIS (scaled to 1,000 pelanggan):             ║
║  - Pelanggan teridentifikasi berisiko: {detected_churners:>4}                  ║
║  - Estimasi berhasil dipertahankan:   {retained_customers:>4} (35% retention)   ║
║  - Revenue diselamatkan: Rp {revenue_saved:>15,}          ║
║  - Biaya intervensi:     Rp {intervention_cost:>15,}          ║
║  - Net benefit:          Rp {net_benefit:>15,}          ║
║                                                                   ║
║  KEY INSIGHTS:                                                    ║
║  1. Pelanggan Month-to-Month memiliki risiko churn tertinggi      ║
║     → Prioritaskan upsell ke One Year / Two Year contract         ║
║  2. Pelanggan baru (tenure < 6 bulan) paling rentan churn         ║
║     → Intensifkan onboarding program di 90 hari pertama           ║
║  3. Customer service calls > 3 = sinyal kuat churn                ║
║     → Flag akun dan eskalasi ke retention specialist              ║
║  4. Electronic Check payment = korelasi positif dengan churn      ║
║     → Tawarkan insentif beralih ke autopay (bank transfer/CC)     ║
╚═══════════════════════════════════════════════════════════════════╝
"""
print(insights)

## Tugas Praktikum

1. **Tugas 1 — Feature Engineering**: Buat setidaknya 3 fitur baru dari dataset yang ada:
   - `avg_monthly_charge` = total_charges / tenure
   - `service_density` = num_services / tenure
   - `high_caller` = 1 jika customer_service_calls > 3, else 0
   
   Latih ulang semua model dan bandingkan apakah fitur baru meningkatkan performa.

2. **Tugas 2 — Handling Class Imbalance**: Jika churn rate rendah (< 20%), model bisa
   bias terhadap kelas mayoritas. Implementasikan:
   - **SMOTE** (oversampling minority class menggunakan `imblearn`)
   - **class_weight='balanced'** pada Logistic Regression dan Random Forest
   
   Bandingkan Recall dan F1-score sebelum dan sesudah.

3. **Tugas 3 — Hyperparameter Tuning**: Gunakan `GridSearchCV` untuk tuning
   Random Forest dengan parameter: `n_estimators` ∈ {50,100,200},
   `max_depth` ∈ {5,8,10,None}, `min_samples_split` ∈ {2,5,10}.
   Gunakan `scoring='recall'` (karena recall lebih penting untuk churn detection).

4. **Tugas 4 — Model Interpretability**: Gunakan library **SHAP** untuk:
   - Plot SHAP summary plot (fitur mana paling berpengaruh global)
   - Plot SHAP waterfall untuk 3 pelanggan individual (1 high-risk, 1 medium-risk, 1 low-risk)
   - Interpretasikan mengapa model memprediksi churn untuk setiap pelanggan